In [1]:
import os
from itertools import product, permutations
import subprocess
import time

### Generation Queens ipynb

#### Main class

In [2]:
class generation_Queens:
    
    def __init__(self, boardsize,  nb_restriction = None):
        self.boardsize = boardsize
        self.nb_queens = boardsize
        self.ymax = boardsize
        self.xmax = boardsize
        self.ymin = 1
        self.xmin = 1
        
        if(nb_restriction):
        
            if(nb_restriction == 1):
                self.positions = self.generate_all_positions_restriction1()
            if(nb_restriction == 2):
                self.positions = self.generate_all_positions_restriction2()
                 
    def pred_open(self, modif1=None, modif2=None):

        _modif1 = (
            str(modif1) if modif1 is not None and modif1 < 0 else "+" + str(modif1)
        )
        _modif2 = (
            str(modif2) if modif2 is not None and modif2 < 0 else "+" + str(modif2)
        )

        if modif1 is None and modif2 is None:
            return "open(?x,?y)"
        elif modif1 is None and modif2 is not None:
            return "open(?x,?y" + _modif2 + ")"
        elif modif1 is not None and modif2 is None:
            return "open(?x" + _modif1 + ",?y)"
        elif modif1 is not None and modif2 is not None:
            return "open(?x" + _modif1 + ",?y" + _modif2 + ")"

        assert "Error in open function, not case matched"

    def generation_preconditions_pos_queen(self, pos_x, pos_y):        
        str_return = []

        # top / down
        for _modif in range(self.ymin, self.ymax + 1, 1):
            # top
            if pos_y + _modif <= self.ymax:
                str_return.append(self.pred_open(None, _modif))
            # down
            if pos_y - _modif >= self.ymin:
                str_return.append(self.pred_open(None, -_modif))

        # left / right
        for _modif in range(self.xmin, self.xmax + 1, 1):
            # right
            if pos_x + _modif <= self.xmax:
                str_return.append(self.pred_open(_modif, None))
            # left
            if pos_x - _modif >= self.xmin:
                str_return.append(self.pred_open(-_modif, None))

        # diagnoal : the only relevant cases to check are the following
        # +-----+----+-----+
        # | TL  |    | TR  |
        # +-----+----+-----+
        # |     | x  |     |
        # +-----+----+-----+
        # | BL  |    | BR  |
        # +-----+----+-----+

        for _modif in range(self.xmin, self.xmax + 1, 1):
            # TL : top left
            if pos_x - _modif >= self.xmin and pos_y + _modif <= self.ymax:
                str_return.append(self.pred_open(-_modif, _modif))
            # BL : bottom left
            if pos_x - _modif >= self.xmin and pos_y - _modif >= self.ymin:
                str_return.append(self.pred_open(-_modif, -_modif))
            # TR : top right
            if pos_x + _modif <= self.xmax and pos_y + _modif <= self.ymax:
                str_return.append(self.pred_open(_modif, _modif))
            # BR : bottom right
            if pos_x + _modif <= self.xmax and pos_y - _modif >= self.ymin:
                str_return.append(self.pred_open(_modif, -_modif))

        return str_return
     
    def generate_function(self, pos_x, pos_y, color):

        preconditions = self.generation_preconditions_pos_queen(pos_x, pos_y)

        str_function = ":action setQueen" + str(pos_x) + str(pos_y) + "\n"
        str_function += ":parameters (?x,?y)\n"
        str_function += ":precondition (" + " ".join(preconditions) + ")\n"
        str_function += ":effect (" + color + "(?x,?y))"

        return str_function
    
    def logical_or(self, a, b):
        return "not(not(" + a + ") not(" + b + "))"

    def generation_preconditions(self):
        str_functions = []

        for i in range(self.xmin, self.xmax + 1, 1):
            for j in range(self.ymin, self.ymax + 1, 1):
                str_functions.append(self.generation_preconditions_pos_queen(i, j))

        return str_functions

    def generation_goal_states(self):
        str_goal = []

        for i in range(self.xmin, self.xmax + 1, 1):
            for j in range(self.ymin, self.ymax + 1, 1):
                str_goal.append("black(" + str(i) + "," + str(j) + ")")

        return str_goal

    def generate_all_positions_restriction1(self):
        all_positions = list(
            product(range(self.boardsize), repeat=self.boardsize)
        )  # 4^4 possibilities

        valid_positions_xy = []
        for pos in all_positions:
            for x, y in enumerate(pos):
                valid_positions_xy.append((x + 1, y + 1))  # Convert to 1-based index
                
        return list(set(valid_positions_xy))

    def generate_all_positions_restriction2(self):

        row_permutations = permutations(range(1, self.boardsize + 1))  # Generate row permutations (1-based index)
    
        # Use a set to store unique (x, y) positions
        unique_positions = set()
        for row_placement in row_permutations:
            for col, row in enumerate(row_placement):
                unique_positions.add((col + 1, row))  # Store (x, y) positions

        return list(unique_positions)

    def generate_domain(self):
        str_domain = ""

        # blackactions
        str_domain += "#blackactions\n"

        for x, y in self.positions:
            str_domain += self.generate_function(x, y, "black")
            str_domain += "\n"

        # whiteactions
        str_domain += "#whiteactions\n"

        function_white = ":action doNothing\n"
        function_white += ":parameters (?x, ?y)\n"
        function_white += ":precondition (open(?x,?y))\n"
        function_white += ":effect (white(?x,?y))\n"

        str_domain += function_white
        
        with open(f"./tmp/domain_{self.boardsize}x{self.boardsize}.ig", "w", encoding='utf-8') as file:
                file.write(str_domain)

    def generate_problem(self, problem_filename, Q1, Q2 = None, Q3 = None):
        
        str_problem = ""
        str_problem += "#boardsize\n"
        str_problem += str(self.boardsize) + " " + str(self.boardsize) + "\n"
        str_problem += "#depth\n"
        str_problem += str(self.boardsize) + "\n"
       
        str_problem += "#times\n"
        str_problem += 't1\n'
        
        str_problem += "#init\n"
        str_problem += f'black({Q1[0]},{Q1[1]})'
        str_problem += f' black({Q2[0]},{Q2[1]})' if Q2 != None else ""
        str_problem += f' black({Q3[0]},{Q3[1]})' if Q3 != None else ""
        
        str_problem += "\n"
        str_problem += "#blackgoal\n"
        
        known_positions = [q for q in [Q1, Q2, Q3] if q is not None]
        nb_postions = len(known_positions)
        remaining_positions = list(set(self.positions) - set(known_positions))
        
        if nb_postions == 3:
            # 13 remaining options 
            for(x,y) in remaining_positions:
                str_problem += f'black({Q1[0]},{Q1[1]}) black({Q1[0]},{Q1[1]}) black({Q3[0]},{Q3[1]}) black({x},{y})\n'  
        elif nb_postions == 2:
            # 91 remaining options 
            for(x,y) in remaining_positions: 
                str_problem += f'black({Q1[0]},{Q1[1]}) black({Q2[0]},{Q2[1]}) black({x},{y})\n' 
        elif nb_postions == 1:
                # 455 reamining options 
                for(x,y) in remaining_positions: 
                    str_problem += f'black({Q1[0]},{Q1[1]}) black({x},{y})\n'            

        # TODO : figure out a way to know how many black Qs there are on the board
        
        str_problem += "#whitegoal\n"
        str_problem += "white(3,1)"

        filename = f"./tmp/problem_{self.boardsize}x{self.boardsize}.ig" if problem_filename is None else problem_filename
        with open(f"./tmp/{filename}.ig", "w", encoding='utf-8') as file:
                file.write(str_problem)
    

In [3]:
def run_q_sage(problem, domain):
    # run Q-Sage.py with the appropriate files + arguments
    project_root = (
        subprocess.run(["pwd"], capture_output=True, text=True)
        .stdout[:-2]
        .split("/")[:-1]
    )
    project_root = "/".join(project_root)

    args = [
        "--game_type",
        "general",
        "--ib_problem",
        problem,
        "--ib_domain",
        domain,
        "-e",
        "ib",
        "--run",
        "2",
        "--encoding_out",
        "intermediate_files/encoding/NQueens.txt",
    ]
      
    result = subprocess.run(
        ["python", "Q-sage.py"] + args,
        cwd=project_root,
        capture_output=True,
        text=True
    )
    
    # processing result 
    result = result.stdout
    i = result.find('setQueen')
     
    if(i != -1):
        l = len('setQueen')
        return (int(result[i+l]), int(result[i+l+1]))
    else:
        return None


In [4]:
def visualise_boad(boardsize, black_queens):
    # Create a board with a border
    board = [['.' for _ in range(boardsize)] for _ in range(boardsize)]
    
    # Place the queens on the board
    qs = [q for q in black_queens if q is not None]
    for x, y in qs:
        board[x-1][y-1] = 'Q'
    
    # Print the board in reverse so that (1,1) is bottom left 
    for row in reversed(board):
        print(" ".join(row))

In [5]:
board_size = 4

Q = generation_Queens(board_size, 2)
Q.generate_domain()

pos1= (2,2)
Q.generate_problem('problem_4x4_incorrect_t1', pos1)
pos2 = run_q_sage("NQueens/tmp/problem_4x4_incorrect_t1.ig", "NQueens/tmp/domain_4x4.ig") 

Q.generate_problem('problem_4x4_incorrect_t2', pos1,pos2)
pos3 = run_q_sage("NQueens/tmp/problem_4x4_incorrect_t2.ig", "NQueens/tmp/domain_4x4.ig") 

Q.generate_problem('problem_4x4_incorrect_t3', pos1,pos2,pos3)
pos4 = run_q_sage("NQueens/tmp/problem_4x4_incorrect_t3.ig", "NQueens/tmp/domain_4x4.ig") 


incorrect_result = [pos1, pos2, pos3, pos4] 
if(None in incorrect_result):
    print("One of the positions could not be found")
else : 
    print("The positions of the queens are : ", incorrect_result)

visualise_boad(board_size, [pos1, pos2, pos3, pos4])

One of the positions could not be found
Q . . .
. . . Q
. Q . .
. . . .


In [6]:
# generalize to function 
def solve_nqueens(Q, pos1, prefix ):

    domain_file = f"NQueens/tmp/domain_{Q.boardsize}x{Q.boardsize}.ig"
    problem_file = f"NQueens/tmp/problem_{Q.boardsize}x{Q.boardsize}_"+prefix
    name = f"problem_{Q.boardsize}x{Q.boardsize}_"+prefix

    Q.generate_problem(name+'_t1', pos1)
    pos2 = run_q_sage(f"{problem_file}_t1.ig", domain_file) 

    Q.generate_problem(name+'_t2', pos1,pos2)
    pos3 = run_q_sage(f"{problem_file}_t2.ig", domain_file) 

    Q.generate_problem(name+'_t3', pos1,pos2,pos3)
    pos4 = run_q_sage(f"{problem_file}_t3.ig", domain_file) 

    result = [pos1, pos2, pos3, pos4] 
    if(None in result):
        print("One of the positions could not be found", result)
        visualise_boad(Q.boardsize, result)
        return False 
    else : 
        print("The positions of the queens are : ", result)
        visualise_boad(Q.boardsize,result)
        return result 


In [7]:
Q = generation_Queens(4, 2)
Q.generate_domain()

solve_nqueens(Q, (1,3), 'correct')

solve_nqueens(Q, (2,2), 'incorrect')

The positions of the queens are :  [(1, 3), (2, 1), (3, 4), (4, 2)]
. Q . .
. . . Q
Q . . .
. . Q .
One of the positions could not be found [(2, 2), (3, 4), (4, 1), None]
Q . . .
. . . Q
. Q . .
. . . .


False

In [15]:
# generating all possible solutions for a board 
def generate_all_solutions(board_size, nb_restriction,root):
    
    Q = generation_Queens(board_size, nb_restriction)
    Q.generate_domain()
    
    positions = [(x, y) for x in range(1, board_size + 1) for y in range(1, board_size + 1)]

    succes = []
    for i, pos1 in enumerate(positions):
            print(f"Solution {i+1} : {pos1}")
            r = solve_nqueens(Q, pos1, f"all_{i+1}")
            print(r)
            if(r != False):
                if any(set(r) == set(existing) for existing in succes):
                    continue
                else : 
                    succes.append(r)
            else : 
                os.remove(f"{root}/NQueens/tmp/problem_{board_size}x{board_size}_all_{i+1}_t1.ig")
                os.remove(f"{root}/NQueens/tmp/problem_{board_size}x{board_size}_all_{i+1}_t2.ig")
                os.remove(f"{root}/NQueens/tmp/problem_{board_size}x{board_size}_all_{i+1}_t3.ig")
            
    return succes

In [9]:
project_root = (
        subprocess.run(["pwd"], capture_output=True, text=True)
        .stdout[:-2]
        .split("/")[:-1]
    )
project_root = "/".join(project_root)
project_root

'/home/annemerel/consciseqQBFencoding'

In [16]:
%%capture captured_output

start_time = time.time()
s = generate_all_solutions(4, 2, project_root)
end_time = time.time()

execution_time = end_time - start_time
minutes, seconds = divmod(execution_time, 60)
with open(f"{project_root}/NQueens/tmp/solution_4x4_output.txt", "w") as file:
    file.write(f"Execution time: {int(minutes)} minutes and {seconds:.2f} seconds\n")
    file.write(f"Solutions: {s}\nOutput: \n")
    file.write(captured_output.stdout)


In [17]:
%%capture captured_output

start_time = time.time()
s = generate_all_solutions(5, 2, project_root)
end_time = time.time()

execution_time = end_time - start_time
minutes, seconds = divmod(execution_time, 60)
with open(f"{project_root}/NQueens/tmp/solution_5x5_output.txt", "w") as file:
    file.write(f"Execution time: {int(minutes)} minutes and {seconds:.2f} seconds\n")
    file.write(f"Solutions: {s}\nOutput: \n")
    file.write(captured_output.stdout)


#### Application
Creating the instance of a boardsize queen problem, can be done by calling the constructor with either one of the restrictions. 
- restriction 1 : Queens can't be in the same column
- restriction 2 : Queens can't be in the same column + row 

In [ ]:
Q = generation_Queens(4,2)

#### Test (debugging purposes)

In [ ]:
Q1 = generation_Queens(4)
unique_positions = Q1.generate_all_positions_restriction2()
flattened_positions = [pos for pos in unique_positions]

# Check if all positions are unique
assert len(flattened_positions) == len(set(flattened_positions)), "Positions are not unique"
# Check if all positions are encoded as (x, y)
assert all(isinstance(pos, tuple) and len(pos) == 2 for pos in flattened_positions), "Not all positions are encoded as (x, y)"

print("All positions are unique and the flattened list is:")
print(flattened_positions)

In [ ]:
Q2 = generation_Queens(4)
unique_positions = Q2.generate_all_positions_restriction1()
flattened_positions2 = [pos for pos in unique_positions]

# Check if all positions are unique
assert len(flattened_positions2) == len(set(flattened_positions2)), "Positions are not unique" + str(flattened_positions2[0:10])
assert all(isinstance(pos, tuple) and len(pos) == 2 for pos in flattened_positions2), "Not all positions are encoded as (x, y)"


print("All positions are unique and the flattened list is:")
print(flattened_positions2)

In [19]:
#  Test if the run_q_sage finds an valid answer 
assert run_q_sage("NQueens/tmp/problem_4x4_3correct.ig", "NQueens/tmp/domain_4x4.ig") == (4,3) , "Error in run_q_sage function with 3 known positions"
assert run_q_sage("NQueens/tmp/problem_4x4_2correct.ig", "NQueens/tmp/domain_4x4.ig") == (3,1), "Error in run_q_sage function with 2 known positions"
assert run_q_sage("NQueens/tmp/problem_4x4_1correct.ig", "NQueens/tmp/domain_4x4.ig") == (2,4), "Error in run_q_sage function with 1 known position"

Result of Q-Sage.py : Start time: 2025-02-12 19:34:45.914150
Git commit hash: b'88de665'
Namespace(version=False, ib_domain='NQueens/tmp/domain_4x4.ig', ib_problem='NQueens/tmp/problem_4x4_3correct.ig', problem='testcases/winning_testcases_ungrounded_new_boards/hein_04_3x3-05.pg', planner_path='/home/annemerel/consciseqQBFencoding', depth=3, xmax=4, ymax=4, ignore_file_depth=0, ignore_file_boardsize=0, e='ib', game_type='general', goal_length=3, run=2, encoding_format=2, encoding_out='intermediate_files/encoding/NQueens.txt', intermediate_encoding_out='intermediate_files/intermediate_encoding', certificate_out='intermediate_files/certificate', solver=2, solver_out='intermediate_files/solver_output', debug=0, run_tests=0, qcir_viz=0, viz_testing=0, viz_meta_data_out='intermediate_files/viz_meta_out', seed=0, renumber_positions=0, restricted_position_constraints=0, black_move_restrictions=1, black_overwriting_black_enable=1, forall_move_restrictions='none', remove_unreachable_nodes=0, ti

In [ ]:

Q_pos = generation_Queens(4)
r = Q_pos.generation_preconditions_pos_queen(2, 2)
r1 = Q_pos.generation_preconditions_pos_queen(1, 1)
r2 = Q_pos.generation_preconditions_pos_queen(3, 2)

assert len(r) == 11, "Length should be 11 but is {}".format(len(r), print(r))
assert len(r1) == 9, "Length should be 9 but is {}".format(len(r1), print(r1))
assert len(r2) == 11, "Length should be 11 but is {}".format(len(r2), print(r2))
print("Tests passed")

r_all = []
for i in range(1, 5):
    for j in range(1, 5):
        r_all.append(Q_pos.generation_preconditions_pos_queen(i, j))

# Flatten the nested list r_all
flattened_r_all = [item for sublist in r_all for item in sublist]

# Check the number of clauses
num_clauses = len(flattened_r_all)
total_of_clauses = 12 * 9 + 4 * 11
assert total_of_clauses == num_clauses, (
    "Total of clauses should be "
    + str(total_of_clauses)
    + " but is {}".format(num_clauses)
)